#
\iftrue
$$
    \renewcommand{\NN}{\mathbf{N}}
    \renewcommand{\RR}{\mathbf{R}}
    \renewcommand{\ZZ}{\mathbf{Z}}
    \renewcommand{\TT}{\mathbf{T}}
    \renewcommand{\RRN}{\RR_N [x]}
    \renewcommand{\ZZN}{\ZZ_N [x]}
    \renewcommand{\TTN}{\TT_N [x]}
    \renewcommand{\BB}{\mathbf{B}}
    joli
$$
\fi
Comandos


## Encripción Completamente Homomorfa con Polinomios sobre el Toro


En este proyecto, queremos presentar lo siguiente: 
- Bases teórica y descripciones en general de FHE
- Funcionamiento teórico de la implementación de TFHE que usamos
- Entrenamiento de un modelo de aprendizaje SENCILLO sobre datos encriptados con este modelo
- Análisis de eficacia y precisión del modelo

--- 

### Funcionamiento teórico de la implementación:

**Preliminares:**

Usaremos las siguientes definiciones en el resto del proyecto:
- $\mathbf{R}$ y $\mathbf{Z}$ denotan los $\mathbf{Z}$-módulos de los reales y los enteros respectivamente
- $ \mathbf{T} = \mathbf{R} / \ZZ$, los reales módulo 1, denota el toro unidimensional (el círculo) como $\ZZ$-módulo
- Sean $N$ una potencia de $2$ dada. $\Phi(x) = x^N +1$, irreducible sobre $\ZZ$
- Denotamos los cocientes $\RRN = \RR[x] / \Phi(x), \ZZN = \ZZ[x] / \Phi(x)$ y $\TTN = \TT[x] / \Phi(x) = \RRN / \ZZN $ vistos como $\ZZN$-módulos
- $\BB = \{0,1 \}$

El sistema de encripción que usamos se basa sobre los siguientes problemas:

**P1: LWE Sobre el Toro**
Dados $n \in \NN$, $\chi$ una distribución sobre $\RR$, y $\mathbf{s} =(s_1,...,s_n)  \xleftarrow{\$} \BB^n $, el problema es distinguir las siiguientes distribuciones:
- $\mathcal{D}_0 = \{ (\mathbf{a},r) : \mathbf{a} \xleftarrow{\$} \TT^n, r \xleftarrow{\$}\TT  \}$
- $\mathcal{D}_1 = \{ (\mathbf{a},r) : \mathbf{a} = (a_1,...,a_n) \xleftarrow{\$} \TT^n, r = \sum_{i=1}^{n} a_i \cdot s_i + e  \}$, con $e \xleftarrow{\chi} \RR$

Básicamente, es difícil distinguir elementos $r \in \TT$ verdaderamente aleatorios, de elementos construidos como cierta combinación lineal de otros elementos del toro (las componentes de $\mathbf{a}$) junto con cierto residuo $e$, incluso teniendo acceso a $\mathbf{a}$

## INSERTAR DIBUJO DE ESTO

Ahora, como esto se va a trabajar dentro de un computador, necesitamos  cierta precisión decimal, que va a ser fija. Si tenemos $\Omega$ bits de precisión, esencialmente estamos reemplazando a $\TT$ por el siguiente módulo:
$$
\hat \TT = \{ t = \sum_{i=0}^\Omega 2^{-i} \cdot t_i \quad \text{mod} \quad 1 | t_i \in \BB  \} = 2^{-\Omega} \ZZ / \ZZ   
$$
Llamemos $q = 2^{-\Omega}$. Podemos tomar representantes $\hat \mu \in \ZZ_q$ de elementos de $\mu \in \hat TT$, tales que $\mu = q^{-1} \hat \mu \quad mod \quad 1$. Por ejemplo, usando $\Omega = 4$, tenemos lo siguiente:
$$
\begin{align*}
    &q = 16 \\
    &\hat \TT = \frac{1}{16} \ZZ / \ZZ &&= \left\{ \mu =  \frac{t_1}{2} + \frac{t_2}{4} + \frac{t_3}{8} + \frac{t_4}{16} \quad \text{mod} \quad : t_i \in \BB \right\}\\
   & &&= \left\{ \mu = \frac{1}{16}[ 8t_1 + 4t_2 + 2t_3 + t_4] \quad \text{mod} \quad : t_i \in \BB \right\{ \\
   & &&= \left\{ \mu = \frac{1}{16} \hat \mu : \hat \mu \in \ZZ_{16} \right\}
\end{align*}
$$
Y podemos entonces codificar en total $q = 16$ números. Usualmente se toma $\Omega = 32$ o $\Omega = 64$.

Para poder añadir espacio donde acomodar el ruido que esconderá los textos, e implementar ciertas operaciones homomorfas, necesitamos establecer ciertos bits significativos $ \hat \omega $ a cero, y almacenamos realmente usando $\omega$ bits de precisión, donde $\hat \omega + \omega \leq \Omega $. Con esto obtenemos la asiguiente representación para elementos del toro:
## ARREGLAR
$$
\bar{\mu} = \underbrace{\overbrace{00001}^{\bar{\omega} \text{ bits}}}_{\text{Parte alta } X_H} \underbrace{\overbrace{v}^{\omega \text{ bits}} \overbrace{000}^{\Omega-\bar{\omega}-\omega \text{ bits}}}_{\text{Parte baja } X_L}
$$

LLamemos $P = 2^{\omega + \hat \omega}$, que corresponde a la cantidad de $\hat X_H$ posibles, y representamos $\hat \mu$ como $\hat \mu = 2 ^{\Omega - (\omega + \hat \omega)} \cdot ( \hat v \quad mod \quad 2^\omega) $
Si $\hat x \in \ZZ_q$, entonces $\hat x  = \frac{q}{p} \hat x_H + \hat x_L, \quad \frac{-q}{2p} \leq \hat x_L \leq \frac{q}{2p}$. $\hat X_L$ se comporta como el ruido de las encripciones, por lo que interesa definir $Upper(\frac{q}{p} \hat x_H + \hat x_L) = \frac{q}{p} \hat x_H + 0$ para reiniciar el ruido. En términos explícitos, 
$$
Upper(\hat x) = \frac{q}{p} \left \lfloor \frac{p}{q} lift(\hat x) \right\rceil \quad mod \quad q 
$$


Con esto, si tenemos un punto $\mu \in \TT$, podemos esconderlo agregandole ruido de la siguiente forma:
1) Fijamos $n \in \NN$, y tomamos una llave aleatoria $(\mathbf{s} = (s_1,...,s_n)) \in \BB^n $ p 
2) Tomamos un vector $\mathbf{a} = (a_1, ... , a_n) \in \TT^n$, y cierto error $e \leftarrow \mathcal{N}(0,\sigma)$, construimos 